# Demonstration of Measurement
You'll using a a 4D time series in this notebook. Along the way, you'll learn the fundamentals of image segmentation, object labeling, and morphological measurement. 

This demo is a jupyter notebook, i.e. intended to be run step by step.

Author: Eric Einspänner
<br>
Contributor: Nastaran Takmilhomayouni

First version: 6th of July 2023


Copyright 2023 Clinic of Neuroradiology, Magdeburg, Germany

License: Apache-2.0

## Table of contents
0. [Initial Set-Up for Google Colab](#initial-set-up-for-google-colab)
1. [Initial Set-Up (offline)](#initial-set-up-offline)
2. [Load image/volume](#Load-the-Image/Volume)
3. [Segmentation](#Segmentation)
    - [Exercise (Segmentation)](#exercise-segmentation)
4. [Select objects](#Select-Objects)
5. [Extract objects](#Extract-Objects)
6. [Measure variance](#Measure-Variance)
7. [Separate histograms](#Separate-Histograms)
8. [Calculate distances](#Calculate-Distance)
9. [COM](#Center-of-Mass-COM)


## Initial Set-Up for Google Colab
<u> Execute these code blocks just in Google Colab! </u>

In [ ]:
!git clone https://github.com/University-Clinic-of-Neuroradiology/python-bootcamp.git

In [ ]:
import os
import sys
from google.colab import output
output.enable_custom_widget_manager()

sys.path.insert(0,'/content/python-bootcamp/notebooks/ImageAnalysis')
os.chdir(sys.path[0])

In [ ]:
%pip install -q ipympl numpy matplotlib imageio pydicom SciPy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import imageio
import scipy.ndimage as ndi
import pydicom
from pydicom.data import get_testdata_file

## Initial Set-Up (offline)

In [ ]:
# Make sure figures appears inline and animations works
# Edit this to ""%matplotlib notebook" when using the "classic" jupyter notebook interface
%matplotlib widget

In [ ]:
# Initial imports etc
import numpy as np
import matplotlib.pyplot as plt

import imageio
import scipy.ndimage as ndi
import pydicom
from pydicom.data import get_testdata_file

## --- Start notebook ---

The following function `format_and_render_plot()` is just a simplify formatting method for the plots in this notebook:

In [ ]:
def format_and_render_plot(axis=False, legend=False):
    '''
    Custom function to simplify common formatting operations for exercises. Operations include: 
    1. Turning off axis grids and legends, if not explicitly requested.
    2. Calling `plt.tight_layout` to improve subplot spacing.
    3. Calling `plt.show()` to render plot.
    '''
    fig = plt.gcf()
    for ax in fig.axes:
        if not axis:
            ax.axis('off')
        if legend:  
            ax.legend(loc='center right')  
    plt.tight_layout()
    plt.show()

## Load the Image/Volume

In [ ]:
# Load the directory (volume)
folder_path = 'Data/Brain/SE000001/MR000000'

# Load the volume
vol = imageio.volread(folder_path)
print(vol.shape)

# save the middle slice as separat image
middle_slice = vol.shape[0] // 2            # // is floor division
im = vol[middle_slice,:,:]

In [ ]:
# min-max normalisation
im_old = im                                     # save original image for later
im = (im - im.min()) / (im.max() - im.min())    # normalise the image, range 0 - 1

# Print the image's data type, minimum and maximum intensity
print('Data type:', im.dtype)
print('Min. value:', im.min())
print('Max value:', im.max())

# Plot the grayscale images
fig, axes = plt.subplots(1, 2)
axes[0].imshow(im_old, cmap='gray')
axes[0].set_title('without normalization', fontweight ="bold")
axes[1].imshow(im, cmap='gray')
axes[1].set_title('with normalization', fontweight ="bold")
format_and_render_plot()

## Segmentation
In this chapter, we'll work with magnetic resonance (MR) imaging data. The full image is a 3D time series spanning.

We start by smoothing our image so that we get a better result when we subsequently segment it. We use `ndi.binary_closing()` to fill any gaps in our mask. At the end we want to label the features of our mask.

In [ ]:
# Smooth intensity values
im_filt = ndi.median_filter(im, size=3)         # size = 3 means 3x3x3 neighbourhood

# Select high-intensity pixels
mask_start = np.where(im_filt > 0.3, 1, 0)      # mask_start is a boolean array
mask = ndi.binary_closing(mask_start)           # fill holes

# Label the objects in "mask"
labels, nlabels = ndi.label(mask)               # labels: each object has a unique number, nlabels: number of objects
print('Num. Labels:', nlabels)

In [ ]:
# Create a `labels` overlay
overlay = np.where(labels > 0, labels, np.nan)

# Use imshow to plot the overlay
fig, axes = plt.subplots(1, 2)
axes[0].imshow(im, cmap='gray')
axes[0].set_title('original', fontweight ="bold")
axes[1].imshow(im, cmap='gray')                             # show image first
axes[1].imshow(overlay, cmap='rainbow', alpha=0.6)          # show overlay second; alpha controls transparency
axes[1].set_title('with segmentation', fontweight ="bold")
format_and_render_plot()

### Exercise (Segmentation)
Try some other paramters and observe the changes. Use new names for `im_filt`, `mask_start`, `mask`, `overlay` and the `labels`!

In [ ]:
# Write your code here






In [ ]:
### Solution - just an example, try your own
# Smooth intensity values
new_im_filt = ndi.median_filter(im, size=4)

# Select high-intensity pixels
new_mask_start = np.where(new_im_filt > 0.35, 1, 0)
new_mask = ndi.binary_closing(new_mask_start, iterations=2)

# Label the objects in "mask"
new_labels, new_nlabels = ndi.label(new_mask)
print('Num. Labels:', new_nlabels)

# Create a `labels` overlay
new_overlay = np.where(new_labels > 0, new_labels, np.nan)

# Use imshow to plot the overlay
fig, axes = plt.subplots(1, 2)
axes[0].imshow(im, cmap='gray')
axes[0].set_title('original', fontweight ="bold")
axes[1].imshow(im, cmap='gray')
axes[1].imshow(new_overlay, cmap='rainbow', alpha=0.6)
axes[1].set_title('with segmentation', fontweight ="bold")
format_and_render_plot()

## Select Objects
Labels are like object "handles" - they give you a way to pick up whole sets of pixels at a time. To select a particular object:

1. Find the label value associated with the object.
2. Create a mask of matching pixels.

In [ ]:
# Label the image "mask"
labels, nlabels = ndi.label(mask)

# Select brain label
brain_val = 2                                           # 2 is the label for the brain (see plot above)
brain_mask = np.where(labels == brain_val, 1, np.nan)   # create brain mask

# Overlay selected label
fig, axes = plt.subplots(1, 2)
axes[0].imshow(im, cmap='gray')
axes[0].set_title('original', fontweight ="bold")
axes[1].imshow(im, cmap='gray')
axes[1].imshow(brain_mask, cmap='rainbow', alpha=0.6)
axes[1].set_title('with segmentation', fontweight ="bold")
format_and_render_plot()

## Extract Objects
Extracting objects from the original image eliminates unrelated pixels and provides new images that can be analyzed independently.

The key is to crop images so that they only include the object of interest. The range of pixel indices that encompass the object is the bounding box.

In [ ]:
brain_mask=brain_mask.astype(np.int64)

# Find bounding box
bboxes =ndi.find_objects(brain_mask)                # returns a list of tuples, each tuple has 3 slices
print('Number of objects:', len(bboxes))            # number of objects found
print('Indices for first box:', bboxes[0])          # print indices for first box

# Crop to index 0
im_brain = im[bboxes[0]]                            # crop the original image to the bounding box

# Plot the cropped image
fig, axes = plt.subplots(1, 2)
axes[0].imshow(im, cmap='gray')
axes[0].set_title('original', fontweight ="bold")
axes[1].imshow(im_brain, cmap='gray')
axes[1].set_title('cropped', fontweight ="bold")
format_and_render_plot()


## Measure Variance
SciPy measurement functions allow you to tailor measurements to specific sets of pixels:

- Specifying `labels` restricts the mask to non-zero pixels.
- Specifying `index` value(s) returns a measure for each label value.

For this exercise, calculate the intensity variance of `vol` with respect to different pixel sets. We have provided the 3D segmented image as `labels`: label 1 is the left ventricle and label 2 is a circular sample of tissue.

In [ ]:
# Variance for all pixels
var_all = ndi.variance(vol, labels=None, index=None)            # labels=None means all pixels, index=None means all objects
print('All pixels:', var_all)

# Variance for labeled pixels
var_labels = ndi.variance(vol, labels, index=None)              # labels=labels means only labeled pixels, index=None means all objects
print('Labeled pixels:', var_labels)

# Variance for each object
var_objects = ndi.variance(vol, labels, index=[1,2])            # labels=labels means only labeled pixels, index=[1,2] means only objects 1 and 2
print('Brain matter:', var_objects[0])
print('Other tissue:', var_objects[1])

## Separate Histograms
A poor tissue segmentation includes multiple tissue types, leading to a wide distribution of intensity values and more variance.

In [ ]:
# Create histograms for selected pixels
hist1 = ndi.histogram(vol, min=0, max=255, bins=256)
hist2 = ndi.histogram(vol, 0, 255, 256, labels=labels)
hist3 = ndi.histogram(vol, 0, 255, 256, labels=labels, index=1)


# Plot the histogram and CDF
fig, axes = plt.subplots(3, 1, sharex=True)                 # sharex=True shares the x-axis between the top and bottom subplot
axes[0].plot(hist1 / hist1.sum(), label='All pixels')
axes[1].plot(hist2 / hist2.sum(), label='All labeled pixels')
axes[2].plot(hist3 / hist3.sum(), label='Brain matter')
format_and_render_plot(axis=True, legend=True)              # axis=True turns on axis grids for the plot; legend=True turns on the legend

## Calculate Distance
A distance transformation calculates the distance from each pixel to a given point, usually the nearest background pixel. This allows you to determine which points in the object are more interior and which are closer to edges.

In this exercise, use the Euclidian distance transform.

In [ ]:
# Calculate distances
brain = np.where(labels == 2, 1, 0)                                             # create brain mask
dists = ndi.distance_transform_edt(brain, sampling=vol.meta['sampling'][1:3])   # calculate distances

# Report on distances
print('Max distance (mm):', ndi.maximum(dists))
print('Max location:', ndi.maximum_position(dists))

# Plot overlay of distances
overlay = np.where(dists > 0, dists, np.nan)                                    # create overlay

fig, axes = plt.subplots(1, 2)
axes[0].imshow(im, cmap='gray')
axes[0].set_title('original', fontweight ="bold")
axes[1].imshow(im, cmap='gray')
axes[1].imshow(overlay, cmap='hot', alpha=0.75)
axes[1].set_title('with distances', fontweight ="bold")
format_and_render_plot()

## Center-Of-Mass (COM)
The distance transformation reveals the most embedded portions of an object. On the other hand, `ndi.center_of_mass()` returns the coordinates for the center of an object.

The "mass" corresponds to intensity values, with higher values pulling the center closer to it.

In [ ]:
# Extract centers of mass for objects 1 and 2
coms = ndi.center_of_mass(vol, labels, index=[1,2])
print('Label 1 center:', coms[0])
print('Label 2 center:', coms[1])

# Add marks to plot
for c0, c1, c2 in coms:
    plt.scatter(c2, c1, s=100, marker='o')
plt.show()